### **<u>Item 67: Use datetime Instead of time for Local Clocks</u>**

- 여러 나라의 Local time을 다뤄야 한다면 time 대신 datetime을 사용할 것을 권장<br><br>
- 여러 다른 시간대를 신뢰할 수 있게 변환하고 싶으면 datetime 과 pytz 모듈을 함께 사용<br><br>
- 항상 시간을 UTC 로 표시하고, 최종적으로 표현하기 직전에 지역시간으로 변환

#### <span style="color: yellowgreen">**_(1) The time Module_**</span>

In [1]:
import time

now = 1552774475
local_tuple = time.localtime(now)
time_format = '%Y-%m-%d %H:%M:%S'
time_str = time.strftime(time_format, local_tuple)
print(time_str)

2019-03-17 07:14:35


In [2]:
time_tuple = time.strptime(time_str, time_format)
utc_now = time.mktime(time_tuple)
print(utc_now)

1552774475.0


&emsp; - **윈도우 환경에서는 시간대를 지원해주지 않는 예시**

In [3]:
import os

if os.name == 'nt':
   print("This example doesn't work on Windows")

else:
    parse_format = '%Y-%m-%d %H:%M:%S %Z'
    depart_sfo = '2019-03-16 15:45:16 PDT'
    time_tuple = time.strptime(depart_sfo, parse_format)
    time_str = time.strftime(time_format, time_tuple)
    print(time_str)

This example doesn't work on Windows


&emsp; - **위의 같은 코드를 리눅스 환경에서 실행했을 때의 모습** <br><br>
<img src="images/linux_time_example.png" width=900> 

&emsp; - **리눅스 환경을 예시로 time 모듈을 통해 여러 시간대를 다루면 위와 같은 ValueError가 발생함** <br><br>
&emsp; - **따라서 이러한 환경과 시간대 지원 차이로 인해 datetime, pytz을 사용할 것이 권장됨**

#### <span style="color: yellowgreen">**_(2) The datetime Module_**</span>

In [4]:
from datetime import datetime, timezone

now = datetime(2024, 9, 13, 10, 13, 4)
now_utc = now.replace(tzinfo=timezone.utc)
now_local = now_utc.astimezone()

print(now_local)

2024-09-13 19:13:04+09:00


&emsp; - **pytz를 이용하면 여러 시간대로 간편하게 전환 가능**

In [5]:
import pytz

arrival_sfo = '2020-08-28 04:13:04'
sfo_dt_naive = datetime.strptime(arrival_sfo, time_format)   # 시간대가 설정되지 않은 시간
eastern = pytz.timezone('US/Pacific')                        # 샌프란시스코의 시간대
sfo_dt = eastern.localize(sfo_dt_naive)                      # 시간대를 샌프란시스코 시간대로 변경
utc_dt = pytz.utc.normalize(sfo_dt.astimezone(pytz.utc))     # UTC로 변경
print(utc_dt)

korea = pytz.timezone('Asia/Seoul')
korea_dt = korea.normalize(utc_dt.astimezone(korea))
print(korea_dt)

nepal = pytz.timezone('Asia/Katmandu')
nepal_dt = nepal.normalize(utc_dt.astimezone(nepal))
print(nepal_dt)

2020-08-28 11:13:04+00:00
2020-08-28 20:13:04+09:00
2020-08-28 16:58:04+05:45


&emsp; - **현재 시간을 다른 로컬 시간으로 변환**

In [10]:
# 한국의 현재 시간 가져오기
korea = pytz.timezone('Asia/Seoul')
korea_dt = datetime.now(korea)
print("한국 현재 시간:", korea_dt.strftime('%Y-%m-%d %H:%M:%S'))

# 샌프란시스코 시간대로 변환
pacific = pytz.timezone('US/Pacific')
sfo_dt = korea_dt.astimezone(pacific)
print("샌프란시스코 현재 시간:", sfo_dt.strftime('%Y-%m-%d %H:%M:%S'))

# 네팔 시간대로 변환
nepal = pytz.timezone('Asia/Katmandu')
nepal_dt = korea_dt.astimezone(nepal)
print("네팔 현재 시간:", nepal_dt.strftime('%Y-%m-%d %H:%M:%S'))

한국 현재 시간: 2024-09-02 16:15:31
샌프란시스코 현재 시간: 2024-09-02 00:15:31
네팔 현재 시간: 2024-09-02 13:00:31


#### <span style="color: yellowgreen">**_Summary_**</span>

&emsp; &#9312; 다른 시간대 간 변환에 time 모듈 사용은 지양 <br><br>
&emsp; &#9313; datetime 내장 모듈과 pytz 커뮤니티 모듈을 사용해 안정적으로 시간을 변환 가능 <br><br>
&emsp; &#9314; 항상 UTC로 시간을 표현